In [46]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import ast
import os
from PIL import Image
import yaml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 경로설정
- pwd
- %cd

12

In [47]:
%cd /content/drive/MyDrive/A_I/Ai

/content/drive/MyDrive/A_I/Ai


In [48]:
# CSV 파일 로드


def load_data_with_pandas(file_path):
    data = pd.read_csv(file_path)
    return data


# 데이터 로드
data_bumper = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/bumper/bumper_data3.csv')
data_door_ed = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/door/door_ed_data3.csv')
data_door_scratch = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/door/door_scratch_data3.csv')
data_fender = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/fender/fender_ed_data3.csv')
data_frame_ed = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_ed_data3.csv')
data_frame_hd = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_hd_data3.csv')
data_frame_sealf = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_sealf_data3.csv')
data_frame_seamf = load_data_with_pandas(
    '/content/drive/MyDrive/A_I/Ai/data/data/output/frame/frame_seamf_data3.csv')

# 데이터 결합
data = pd.concat([
    data_bumper,
    data_door_ed,
    data_door_scratch,
    data_fender,
    data_frame_ed,
    data_frame_hd,
    data_frame_sealf,
    data_frame_seamf
], axis=0)

# data.to_csv('./data/data.csv', index=False)

# 학습 데이터와 검증 데이터로 나누기 (Stratified Sampling)
# y에는 라벨, x에는 나머지 특성 정보
# x = data.drop('quality', axis=1)  # 'label' 컬럼은 실제 클래스 라벨이 들어있는 컬럼
y = data['quality']

# Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
train_data, valid_data = train_test_split(
    data, test_size=0.2, stratify=y, random_state=55)

# CSV로 저장
train_data.to_csv('/content/drive/MyDrive/A_I/Ai/data/data/train_data.csv', index=False)
valid_data.to_csv('/content/drive/MyDrive/A_I/Ai/data/data/valid_data.csv', index=False)

# 결과 출력
print(f"학습 데이터 개수: {len(train_data)}")
print(f"검증 데이터 개수: {len(valid_data)}")

학습 데이터 개수: 5904
검증 데이터 개수: 1477


In [95]:
# Step 1: 학습 데이터에서 20% 추출
train_20, _ = train_test_split(train_data, test_size=1, random_state=35)

# Step 2: 검증 데이터에서 20% 추출
valid_20, _ = train_test_split(valid_data, test_size=1, random_state=35)

# Step 3: 각각의 20%에서 다시 20% 추출 (성능 검증용 데이터 생성)
train_test_20, _ = train_test_split(train_20, test_size=0.8, random_state=42)  # 학습 데이터 20%에서 20% 추출
valid_test_20, _ = train_test_split(valid_20, test_size=0.8, random_state=42)  # 검증 데이터 20%에서 20% 추출
#test_test_20

# CSV로 저장
train_test_20.to_csv('/content/drive/MyDrive/A_I/Ai/data/data/train_test_20.csv', index=False)
valid_test_20.to_csv('/content/drive/MyDrive/A_I/Ai/data/data/valid_test_20.csv', index=False)

In [50]:
import os
import shutil

def normalize_paths(data, path_column):
    """
    데이터프레임의 경로를 정규화하고 슬래시를 통일합니다.

    Args:
        data (pd.DataFrame): 경로가 포함된 데이터프레임
        path_column (str): 경로 열의 이름

    Returns:
        pd.DataFrame: 경로가 정규화된 데이터프레임
    """
    data[path_column] = data[path_column].apply(lambda x: os.path.normpath(x).replace("\\", "/"))
    return data

def copy_images(data, output_dir):
    """
    데이터프레임의 이미지 파일을 지정된 출력 디렉토리로 복사합니다.

    Args:
        data (pd.DataFrame): 이미지 경로가 포함된 데이터프레임
        output_dir (str): 출력 디렉토리 경로
    """
    # 이미지 디렉토리 생성
    image_dir = os.path.join(output_dir, 'images')
    os.makedirs(image_dir, exist_ok=True)

    # 이미지 복사
    for index, row in data.iterrows():
        image_path = row['path']
        try:
            # 파일 존재 여부 확인
            if not os.path.exists(image_path):
                print(f"경고: 파일이 존재하지 않습니다 - {image_path}")
                continue

            # 복사 대상 파일 이름 생성
            destination = os.path.join(image_dir, os.path.basename(image_path))

            # 파일 복사
            shutil.copy(image_path, destination)
        except Exception as e:
            print(f"오류: {image_path} 복사 실패 - {e}")

# 데이터프레임의 경로 정규화 및 슬래시 통일
train_test_20 = normalize_paths(train_test_20, 'path')
valid_test_20 = normalize_paths(valid_test_20, 'path')

# 학습 데이터와 검증 데이터를 각각 처리
copy_images(train_test_20, './train')
copy_images(valid_test_20, './valid')


In [51]:
import os
from PIL import Image


def extract_image_info(image_paths):
    """
    이미지 경로 리스트에서 이미지 크기, 디렉토리 경로, 파일 이름을 추출하는 함수

    Args:
        image_paths (list): 이미지 경로 리스트

    Returns:
        tuple: (image_sizes, image_dirs, file_names)
            image_sizes (list): 이미지 크기 리스트
            image_dirs (list): 이미지 디렉토리 경로 리스트
            file_names (list): 이미지 파일 이름 리스트
    """

    image_sizes = []
    image_dirs = []
    file_names = []

    for path in image_paths:
        try:
            image = Image.open(path)
            width, height = image.size
            image_sizes.append((width, height))
            image_dirs.append(os.path.dirname(path) + os.sep)
            file_names.append(os.path.splitext(os.path.basename(path))[0])
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")

    return image_sizes, image_dirs, file_names


# 데이터 준비 (예시)
train_image_paths = train_test_20.iloc[:, 0].tolist()
valid_image_paths = valid_test_20.iloc[:, 0].tolist()

# 이미지 정보 추출
train_sizes, train_dirs, train_names = extract_image_info(train_image_paths)
valid_sizes, valid_dirs, valid_names = extract_image_info(valid_image_paths)

# 결과 출력
print("Train 이미지 크기:", train_sizes[:5])
print("Train 이미지 디렉토리:", train_dirs[:5])
print("Train 파일 이름:", train_names[:5])
print()
print("Validation 이미지 크기:", valid_sizes[:5])
print("Validation 이미지 디렉토리:", valid_dirs[:5])
print("Validation 파일 이름:", valid_names[:5])

Train 이미지 크기: [(4224, 2376), (4032, 1960), (4032, 1908), (4032, 1908), (4032, 1908)]
Train 이미지 디렉토리: ['data/data/source_data/door/Exterior_damage/', 'data/data/source_data/frame/Seam_failure/', 'data/data/source_data/bumper/scratch/', 'data/data/source_data/door/scratch/', 'data/data/source_data/bumper/scratch/']
Train 파일 이름: ['204_102_20_6d79e1bc-442e-45d6-bd7f-a09baeefee49', '207_212_20_33af8e26-7021-421c-88fe-683c37c5e565', '205_101_20_d7504074-f53e-4c44-84ec-636e61f902ac', '204_101_10_5620de38-ef45-4fb1-a051-4067b7704f1c', '205_101_20_05b4b1a5-6a7a-4e68-bdb5-17316ae94eb4']

Validation 이미지 크기: [(4032, 1908), (4032, 1908), (4032, 1816), (4224, 2376), (2448, 2048)]
Validation 이미지 디렉토리: ['data/data/source_data/door/Exterior_damage/', 'data/data/source_data/frame/Seam_failure/', 'data/data/source_data/frame/Seam_failure/', 'data/data/source_data/door/scratch/', 'data/data/source_data/frame/Hole_distortion/']
Validation 파일 이름: ['204_102_20_019dc27d-2034-44b5-aea2-d519c5209082', '207_212_

In [52]:
import os
from PIL import Image


def extract_image_info(image_paths):
    """
    이미지 경로 리스트에서 이미지 크기, 디렉토리 경로, 파일 이름을 추출하는 함수

    Args:
        image_paths (list): 이미지 경로 리스트

    Returns:
        tuple: (image_sizes, image_dirs, file_names)
            image_sizes (list): 이미지 크기 리스트
            image_dirs (list): 이미지 디렉토리 경로 리스트
            file_names (list): 이미지 파일 이름 리스트
    """

    image_sizes = []
    image_dirs = []
    file_names = []

    for path in image_paths:
        try:
            image = Image.open(path)
            width, height = image.size
            image_sizes.append((width, height))
            image_dirs.append(os.path.dirname(path) + os.sep)
            file_names.append(os.path.splitext(os.path.basename(path))[0])
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")

    return image_sizes, image_dirs, file_names


# 데이터 준비 (예시)
train_image_paths = train_test_20.iloc[:, 0].tolist()
valid_image_paths = valid_test_20.iloc[:, 0].tolist()

# 이미지 정보 추출
train_sizes, train_dirs, train_names = extract_image_info(train_image_paths)
valid_sizes, valid_dirs, valid_names = extract_image_info(valid_image_paths)

# 결과 출력
print("Train 이미지 크기:", train_sizes[:5])
print("Train 이미지 디렉토리:", train_dirs[:5])
print("Train 파일 이름:", train_names[:5])
print()
print("Validation 이미지 크기:", valid_sizes[:5])
print("Validation 이미지 디렉토리:", valid_dirs[:5])
print("Validation 파일 이름:", valid_names[:5])

Train 이미지 크기: [(4224, 2376), (4032, 1960), (4032, 1908), (4032, 1908), (4032, 1908)]
Train 이미지 디렉토리: ['data/data/source_data/door/Exterior_damage/', 'data/data/source_data/frame/Seam_failure/', 'data/data/source_data/bumper/scratch/', 'data/data/source_data/door/scratch/', 'data/data/source_data/bumper/scratch/']
Train 파일 이름: ['204_102_20_6d79e1bc-442e-45d6-bd7f-a09baeefee49', '207_212_20_33af8e26-7021-421c-88fe-683c37c5e565', '205_101_20_d7504074-f53e-4c44-84ec-636e61f902ac', '204_101_10_5620de38-ef45-4fb1-a051-4067b7704f1c', '205_101_20_05b4b1a5-6a7a-4e68-bdb5-17316ae94eb4']

Validation 이미지 크기: [(4032, 1908), (4032, 1908), (4032, 1816), (4224, 2376), (2448, 2048)]
Validation 이미지 디렉토리: ['data/data/source_data/door/Exterior_damage/', 'data/data/source_data/frame/Seam_failure/', 'data/data/source_data/frame/Seam_failure/', 'data/data/source_data/door/scratch/', 'data/data/source_data/frame/Hole_distortion/']
Validation 파일 이름: ['204_102_20_019dc27d-2034-44b5-aea2-d519c5209082', '207_212_

In [53]:
# 부품 상태를 체크하는 함수
def get_label(part, status):
    if part == "범퍼":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "도어":
        if status == 1:
            return 2
        elif status == 0:
            return 3
    elif part == "휀더":
        if status == 1:
            return 4
        elif status == 0:
            return 5
    elif part == "프레임":
        if status == 1:
            return 6
        elif status == 0:
            return 7
    return None  # 기본적으로 반환할 값이 없으면 None


# 예시 부품과 상태
parts_and_status_train = []
parts_and_status_valid = []
for quality, part in zip(train_test_20.iloc[:, 1], train_test_20.iloc[:, 2]):
    parts_and_status_train.append({"part": part, "status": quality})

for quality, part in zip(valid_test_20.iloc[:, 1], valid_test_20.iloc[:, 2]):
    parts_and_status_valid.append({"part": part, "status": quality})

print(f"parts_and_status_train : {parts_and_status_train}")
print(f"parts_and_status_valid : {parts_and_status_valid}")

# 라벨링 결과를 저장할 리스트
labels_train = []
labels_valid = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train:
    label = get_label(item["part"], item["status"])
    labels_train.append(label)
for item in parts_and_status_valid:
    label = get_label(item["part"], item["status"])
    labels_valid.append(label)

# # 결과 출력
print("train 라벨링 결과 :", labels_train)
print("valid 라벨링 결과 :", labels_valid)

parts_and_status_train : [{'part': '도어', 'status': 0}, {'part': '프레임', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '도어', 'status': 1}, {'part': '범퍼', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '도어', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '도어', 'status': 1}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '도어', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 0}, {'part': '도어', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '프레임', 'status': 0}, {'part': '도어', 'status': 0}, {'part': '휀더', 'status': 1}, {'

In [62]:
def convert_to_yolo_format(bbox, label, image_width, image_height):
    yolo_labels = []  # 각 이미지에 대한 라벨을 담을 리스트

    if not bbox:
        # bbox가 비어있으면 기본값으로 라벨을 설정
        yolo_labels.append(f"{label} 0.5 0.5 0.0 0.0")
    else:
        # bbox가 있으면 여러 개의 파손 부위를 처리
        for x_min, y_min, width, height in bbox:
            # YOLO 형식에 맞게 좌표를 정규화
            x_center = (x_min + width / 2) / image_width
            y_center = (y_min + height / 2) / image_height
            norm_width = width / image_width
            norm_height = height / image_height
            # 정규화된 좌표와 함께 라벨을 저장
            yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}")

    return yolo_labels


def save_yolo_labels(output_dir, bboxes, labels, size, names):
    # 각 이미지에 대해 라벨을 텍스트 파일로 저장
    for bbox, label, (w, h), name in zip(bboxes, labels, size, names):
        yolo_labels = convert_to_yolo_format(bbox, label, w, h)

        # 라벨을 .txt 파일로 저장
        txt_file_path = os.path.join(output_dir, f"{name}.txt")
        with open(txt_file_path, 'w') as txt_file:
            for yolo_data in yolo_labels:
                txt_file.write(f"{yolo_data}\n")



# 라벨을 저장할 디렉토리
train_output_dir = './train/labels'
os.makedirs(train_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(train_output_dir, train_bboxes, labels_train, train_sizes, train_names)

# 라벨을 저장할 디렉토리
valid_output_dir = './valid/labels'
os.makedirs(valid_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(valid_output_dir, valid_bboxes, labels_valid, valid_sizes, valid_names)

In [69]:
# YAML 파일에 저장
yaml_data = {
    "train": "/content/drive/MyDrive/A_I/Ai/train/images",
    "val": "/content/drive/MyDrive/A_I/Ai/valid/images",
    "nc": 8,
    "names": ["bumper_ok",
              "bumper_faulty",
              "door_ok",
              "door_faulty",
              "fender_ok",
              "fender_faulty",
              "frame_ok",
              "frame_faulty"]
}


with open(f"data.yaml", "w") as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)

# print("YAML 파일이 생성되었습니다.")

In [57]:
%pip install ultralytics

In [85]:

import os
from ultralytics import YOLO

# 데이터 설정 파일 경로
data_yaml = "data.yaml"

# YOLO 모델 정의 (랜덤 초기화로 학습 시작)
model = YOLO("yolov8m.pt")  # 'yolov5n'은 작은 모델, 필요에 따라 yolov5s, yolov5m 선택 가능
# model.model = model.model.reset_parameters()  # 가중치 초기화 (랜덤 가중치)

# 학습 설정
model.train(
    data=data_yaml,           # 데이터 설정 파일 경로
    epochs=10,               # 학습 에폭 수
    batch=16,            # 배치 크기
    imgsz=640,                # 입력 이미지 크기
    device=0,                 # GPU 사용
    workers=4,                # 데이터 로드 병렬 처리 워커 수
    )

model.save("model.pt")

Ultralytics 8.3.64 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=

train: Scanning /content/drive/MyDrive/A_I/Ai/train/labels.cache... 236 images, 0 backgrounds, 0 corrupt: 100%|██████████| 236/236 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_0a2a187d-ffc9-4358-8b41-da86465cd02d.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_2ba0bfd5-d0c7-41df-859a-286f27badb75.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_2cf1b80f-698e-4027-b508-1d524e9597b3.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_32f5de4b-d17b-44ef-bf80-ccc01e425304.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_51c63e60-6ee7-43d4-b106-2ca3a00f4577.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_5620de38-ef45-4fb1-a051-4067b7704f1c.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/train/images/204_101_10_563d12d6-13da-4b91-8591-474d32b80b9f.jpg: corrupt JP


val: Scanning /content/drive/MyDrive/A_I/Ai/valid/labels.cache... 59 images, 0 backgrounds, 0 corrupt: 100%|██████████| 59/59 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/valid/images/204_101_10_814f2bff-c85f-40b8-89a6-bae9a47350ff.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/valid/images/204_101_10_e67d6a52-d240-4161-b7d4-9e08a53731f4.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/valid/images/204_101_20_d8add5a1-debc-42e7-8dfa-6f5a952f5c82.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/valid/images/204_101_20_e472c8cd-8a2a-4d28-ba01-9be28185eb16.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/valid/images/204_102_10_7ce09908-4cd1-4a6a-8372-dd0edfaa3367.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/valid/images/204_102_10_c1fcfdfa-58a7-41b4-8910-dfb65661e024.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/A_I/Ai/valid/images/204_102_10_d1d9822a-ff0f-42c9-8952-2132179a9fc9.jpg: corrupt JPEG restored an

...

      10/10      6.86G      1.765       2.62      1.718          9        640: 100%|██████████| 15/15 [00:23<00:00,  1.54s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.48it/s]

                   all         59         62      0.401      0.143     0.0679     0.0268



10 epochs completed in 0.106 hours.
Optimizer stripped from runs/detect/train11/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train11/weights/best.pt, 52.0MB

Validating runs/detect/train11/weights/best.pt...
Ultralytics 8.3.64 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,844,392 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.30it/s]


                   all         59         62      0.401      0.143      0.068     0.0268
             bumper_ok          7          7          1          0          0          0
         bumper_faulty          5          8      0.115       0.25     0.0881     0.0307
               door_ok          8          8          0          0          0          0
           door_faulty          6          6      0.164      0.167     0.0866     0.0422
             fender_ok          3          3          0          0          0          0
         fender_faulty          4          4          1          0          0          0
              frame_ok         12         12      0.675     0.0833     0.0921      0.045
          frame_faulty         14         14      0.255      0.643      0.277     0.0963
Speed: 0.2ms preprocess, 8.5ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/train11


In [83]:
model = YOLO("runs/train/custom_model/weights/best.pt")

# 추론
results = model.predict(
    source="/content/drive/MyDrive/A_I/Ai/test_images",  # 테스트 이미지 경로
    save=True,                # 결과 저장
    imgsz=640                 # 입력 이미지 크기
)

# 결과 저장 경로 출력
print(f"Results saved to {results.save_dir}")

FileNotFoundError: [Errno 2] No such file or directory: 'runs/train/custom_model/weights/best.pt'

In [87]:
import os

# 가중치 파일 경로 확인
weights_dir = "runs/train/custom_model/weights"
if os.path.exists(weights_dir):
    print("Weights directory found. Files:")
    print(os.listdir(weights_dir))
else:
    print(f"Weights directory not found: {weights_dir}")

Weights directory not found: runs/train/custom_model/weights


In [96]:
# 모델 로드 및 추론
model = YOLO("/content/drive/MyDrive/A_I/Ai/runs/detect/train11/weights/best.pt") # ex)train11 바꾸기
results = model.predict(source="/content/drive/MyDrive/A_I/Ai/valid/images", save=True, imgsz=640)


image 1/59 /content/drive/MyDrive/A_I/Ai/valid/images/204_101_10_08e9e2fe-6870-4b41-98b9-1812b886e1e6.jpg: 320x640 (no detections), 23.6ms
image 2/59 /content/drive/MyDrive/A_I/Ai/valid/images/204_101_10_0cfc1ec4-c32d-49da-a79f-5cc96979537d.jpg: 480x640 (no detections), 29.1ms
image 3/59 /content/drive/MyDrive/A_I/Ai/valid/images/204_101_10_2922bb55-cd6d-4b1f-b717-d6829c434df8.jpg: 384x640 (no detections), 26.1ms
image 4/59 /content/drive/MyDrive/A_I/Ai/valid/images/204_101_10_814f2bff-c85f-40b8-89a6-bae9a47350ff.jpg: 320x640 (no detections), 23.7ms
image 5/59 /content/drive/MyDrive/A_I/Ai/valid/images/204_101_10_e67d6a52-d240-4161-b7d4-9e08a53731f4.jpg: 480x640 (no detections), 28.9ms
image 6/59 /content/drive/MyDrive/A_I/Ai/valid/images/204_101_20_d8add5a1-debc-42e7-8dfa-6f5a952f5c82.jpg: 288x640 (no detections), 22.8ms
image 7/59 /content/drive/MyDrive/A_I/Ai/valid/images/204_101_20_e472c8cd-8a2a-4d28-ba01-9be28185eb16.jpg: 320x640 (no detections), 24.6ms
image 8/59 /content/drive/